# Parámetros Conocidos (Domain parameters)
- P, número primo que determina el campo sobre el que trabaja la curva $\mathbb{F}_p$
- n es el orden del campo, esto es el número de puntos enteros.
- Acurve y Bcurve representan los parámetros de la ecuación $y^2=x^3 + (Acurve) x^2 + (Bcurve)$
- PrivKey es un entero que se encuentra dentro del intervalo $ 1 \leq (PrivKey) \leq p-1 $

Ejemplo básico:

>Pcurve=17 <br>
N=19 #19 <br>
Acurve=2; Bcurve=2 <br>
Gx=5; Gy=1 <br>
Gpoint=(Gx,Gy) #Definimos una tupla (un par ordenado) <br>
privKey=13 <br>

In [75]:
Pcurve=2**256-2**32-2**9-2**8-2**7-2**6-2**4-1
N=0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEBAAEDCE6AF48A03BBFD25E8CD0364141
Acurve=0; Bcurve=7
privKey=0xA0DC65FFCA799873CBEA0AC274015B9526505DAAAED385155425F7337704883E

Para obetener el punto generado, se nos da el punto $x$ de forma hexadecimal, por lo tanto, convirtiendolo en decimal para obtener la parte $y$: 

In [78]:
Gx=int(0x79BE667EF9DCBBAC55A06295CE870B07029BFCDB2DCE28D959F2815B16F81798)
Gx

55066263022277343669578718895168534326250603453777594175500187360389116729240L

Para tener el punto hacemos $y=\sqrt{x^3+7}$

In [91]:
import math

Gy=int(math.sqrt((Gx*Gx*Gx)+7))
Gy

12921960443297828860891648158292471819675107701313235361247994568418618525728663521040370767731172617416358500499456L

In [92]:
Gpoint=(Gx,Gy)


### Con los parámetros dados, el objetivo es logar generar una llave pública 
$ \Large K_{pub}=K_{priv}G$

La primera función a implementar es el de la división modular
***
 $\frac{a}{b} \mod n = (a * b^{-1}) \mod n$
***
Por tanto, para hacer este producto más sencillo, encontramos el inverso de $b$. 
***
### Algoritmo Extendido  de Euclides 
***
> 
Para determinar el inverso multiplicativo de $b$ se tiene el siguiente algoritmo: <br>
Entrada: a y n 
$r_1 = n , r_2 =a$ <br>
$t_1 =0 , t_2= 1$ <br>
Mientras $r_1>1$ entonces: <br>
$q=\frac{r_1}{r_2}$ (División entera)<br>
$r=r_1-q*r_2$<br>
$r_1=r_2, r_2=r$<br>
$t=t_1-q*t_2$<br>
$t_1=t_2, t_2=t$<br>
Fin Mientras<br>
Si $r_1=1$ entonces $t=a^{-1}$

Tomado de _[MultiInverse](https://cap430.files.wordpress.com/2011/03/multiplicative-inverse.pdf)_

In [33]:
def ECDiv(a, n=Pcurve): #Regresa el inverso de a módulo n
    r1,r2=n,a
    t1,t2=0,1
    while r1 >1:
        q=r1 // r2
        r=r1-q*r2
        r1,r2=r2,r
        t=t1-q*t2
        t1,t2=t2,t    
        
    return t1%n #Este último es para evitar valores negativos

### Ejemplo

In [34]:
print ECDiv(11,26)
print "11*19 mod 26 =" , (ECDiv(11,26)* 11)%26          

19
11*19 mod 26 = 1


# Suma de puntos en una curva elíptica

## Puntos diferentes
Para la suma de puntos $P=(x_P, y_P)$ y $Q=(x_Q, y_Q)$, para calcular $P +Q= R$ donde $P \neq Q$
y $R=(x_R,y_R)$ se obtiene haciendo las siguientes fórmulas: 
***
La pendiente es calculada como $m= \frac{(y_Q- y_P)}{(x_Q-x_Q)} \mod p$
*** 
$ x_R= (m^2-x_P-x_Q) \mod p $ <br>
$ y_R= (m(x_P-x_R)-y_P) \mod p$
***
De donde $a=(a[0], a[1])$ y $b=(b[0], b[1])$, así se intenta hacer $a+b=c=(x,y)$ 
*** 
> Por lo tanto el código tiene la siguiente forma: <br>
$m= (b[1]-a[1])*(b[0]-a[0])^{-1} \mod Pcurve$ <br>
$ x= (m^2-a[0]-b[0]) \mod p $ <br>
$ y= (m(a[0]-x)-a[1]) \mod p$

In [61]:
def ECSum(a,b): #Suma de puntos en la curva elíptica
    m=( ((b[1]-a[1])% Pcurve) * ECDiv((b[0]-a[0])%Pcurve, Pcurve) ) % Pcurve
    x=(m*m-a[0]-b[0]) % Pcurve
    y=(m*(a[0]-x)-a[1]) % Pcurve
    return (x,y)

### Ejemplo

In [36]:
a=(5,1)
b=(6,3)
print ECSum(a,b)

(10, 6)


## Puntos iguales (Doubling)

Para la suma de puntos $P=(x, y)$ , para calcular $P + P= 2P$ se obtiene haciendo las siguientes fórmulas: 
***
La pendiente es calculada como $\displaystyle m= ( \frac{3*x^2+Acurve}{2y} ) \mod p$
*** 
$ x_R= (m^2-x_P-x_Q) \mod p $ <br>
$ y_R= (m(x_P-x_R)-y_P) \mod p$
***
Notemos que en el códgio $a=(a[0], a[1])$ y $b=(b[0], b[1])$ y adecuando las fórmulas anteriores tenemos lo siguiente: 
*** 
 > Código <br>
 $ m= (3*a[0]^2 + Acurve ) (2*a[1])^{-1} \mod p$ <br>
 $ x= (m^2-2*a[0]) \mod p $ <br>
 $ y= (m(a[0]-x)-a[1]) \mod p$ 
 

In [6]:
def ECDoub(a): #Suma iterada del punto 
    m= ((3*(a[0]*a[0])+Acurve)* ECDiv(2*a[1], Pcurve))% Pcurve
    x= ((m*m)-2*a[0]) % Pcurve
    y= (m*(a[0]-x)-a[1]) % Pcurve
    return (x,y)

### Ejemplo

In [37]:
print "(5,1)+(5,1)=", ECDoub((5,1))

(5,1)+(5,1)= (6, 3)


# Add and Doubling
Notemos que el hecho de calcular $R=dQ$ es equivalente a hacer 
$R=dP=P+P+ \cdots + P_{d veces}$ hacemos uso del algoritmo llamado "double and add"
que tiene el siguiente algoritmo:
***
Dado un punto $P$ y un entero $d$ que se puede representar como 
$d=\sum_{i=0}^{t} d_i 2^i $ con $d_i \in \{0,1\}$
***
> 
Inicialización<br>
$Q=P$ <br>
Para cada i de len(d)-1 hasta 0 hacer: <br>
Q= Q+Q
Si d_i=1 entonces : <br>
Q=Q+P <br>
Fin Para

In [87]:
def ECMult(Genpoint, key): #Operación de dQ, Entrada: P , d
    if key ==0 or  key >= N: raise Exception("Llave inválida") #Evitar errores
    keyBin=str( bin(key) ) [2:] #Pasamos d en su representación binaria
    #print "Binario",keyBin
    Q=Genpoint #Inicialización 
    for i in range(1,len(keyBin)):
        #print Q
        Q = ECDoub(Q); #print "Doub",Q
        if keyBin[i] =='1':
            Q=ECSum(Q,Genpoint); #print "Add",Q
       
    return Q 
    

# Diffie Hellman Key Exchange
***
Una vez conocidos los parámetros $\{C,a,b,p,n,G\}$, podemos obtener una llave pública de la siguiente manera:
> $K_{p}=dG$


In [73]:
print "La llave pública es:", ECMult((5,1), 11)

La llave pública es: Binario 1011
Doub (6, 3)
Doub (3, 1)
Add (9, 16)
Doub (7, 11)
Add (13, 10)
(13, 10)


# Ejemplo

De la página gubernamental para la selección de parámetros públicos que se puede visitar dando click 
_[aquí](http://www.secg.org/sec2-v2.pdf)_ tomaremos como **Ejemplo** los siguientes parámetros:
***
> 
$EC: y^2= x^2+ 7$ <br>
$p=2^{256}−2^{32}−2^{9}−2^{28}−2^{7}−2^{6}−2^{4}−1$ <br>
$n=FFFFFFFF FFFFFFFF FFFFFFFF FFFFFFFE BAAEDCE6 AF48A03B BFD25E8C
D0364141$ <br>


In [93]:
print "La llave pública es:", ECMult(Gpoint,privKey)

La llave pública es: (115009754428600178819952671679209779110224617022865186266424642699528293783604L, 57427883689044799006240979316062677995411505093762760627500803341082479000615L)
